In [1]:
# Optional: install if missing (uncomment to run)
import sys
!{sys.executable} -m pip install MetaTrader5 stable-baselines3[extra] gymnasium numpy pandas matplotlib


In [1]:
# Cell 2
import os, glob, time, json
from datetime import datetime
import numpy as np
import pandas as pd
import MetaTrader5 as mt5
import matplotlib.pyplot as plt

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecNormalize


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [2]:
# === PATHS (from your config) ===
DATA_DIR = os.path.join("data", "multiasset")
MODEL_DIR = os.path.join("models", "multiasset")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")

MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
VEC_FILE = os.path.join(MODEL_DIR, "vec_normalize.pkl")
METRICS_FILE = os.path.join(MODEL_DIR, "eval_metrics.json")



In [3]:
# --------------------------------------
# Timeframe mapping
# --------------------------------------
TIMEFRAME = "M15"
TF_MAP = {
    "M1": mt5.TIMEFRAME_M1,
    "M5": mt5.TIMEFRAME_M5,
    "M15": mt5.TIMEFRAME_M15,
    "M30": mt5.TIMEFRAME_M30,
    "H1": mt5.TIMEFRAME_H1,
    "H4": mt5.TIMEFRAME_H4,
    "D1": mt5.TIMEFRAME_D1
}
tf = TF_MAP[TIMEFRAME.upper()]

In [4]:
# Runtime config
WINDOW = 50
TF_MT5 = mt5.TIMEFRAME_M15   # M1 as used earlier
DRY_RUN = False              # KEEP True while testing
DEFAULT_LOT = 0.1           # as requested
MAX_POS_PER_SYMBOL = 2

# SL/TP settings: SL computed from volatility (function below); TP = 3 * SL
DEFAULT_SL_PIPS_FALLBACK = 20
TP_MULT = 3

# Logging
LOG_DIR = os.path.join(MODEL_DIR, "live_logs")
os.makedirs(LOG_DIR, exist_ok=True)
LOG_FILE = os.path.join(LOG_DIR, "live_trade_logs.csv")


In [5]:
# Cell 3
def make_safe_name(sym: str) -> str:
    return sym.replace(" ", "_").replace("/", "_").replace("(", "").replace(")", "").replace(".", "_")

#def raw_from_safe(safe: str) -> str:
 #   return safe.replace("_", " ")

def safe_from_raw(raw: str) -> str:
    return make_safe_name(raw)


In [6]:
# ================================================================
# ⚙️ Cell 4 - Symbols Selection
# ================================================================
SYMBOLS = [
    # --- Volatility Indices (Standard) ---
    "Volatility 10 Index",
    "Volatility 25 Index",
    "Volatility 50 Index",
    "Volatility 75 Index",
    "Volatility 100 Index",

    # --- Volatility 1s Indices ---
    "Volatility 10 (1s) Index",
    "Volatility 25 (1s) Index",
    "Volatility 50 (1s) Index",
    "Volatility 75 (1s) Index",
    "Volatility 100 (1s) Index",

    # --- Volatility 10s Indices ---
    "Volatility 10 (10s) Index",
    "Volatility 25 (10s) Index",
    "Volatility 50 (10s) Index",
    "Volatility 75 (10s) Index",
    "Volatility 100 (10s) Index",

    # --- Jump Indices ---
    "Jump 10 Index",
    "Jump 25 Index",
    "Jump 50 Index",
    "Jump 75 Index",
    "Jump 100 Index",

    # --- Step Indices ---
    "Step Index 25",
    "Step Index 50",
    "Step Index 75",
    "Step Index 100",

    # --- Forex Reference ---
    "EURUSD"
]


In [23]:
def fetch_data_for_symbol(safe_name, window: int = WINDOW):
    """
    Fetch recent bars from MT5 for the broker symbol mapped from safe_name.
    Returns DataFrame with columns open,high,low,close,volume,Close_raw
    """
    mt5.initialize()
    raw_symbol = SYMBOLS
    info = mt5.symbol_info(raw_symbol)
    if info is None:
        print(f"❌ Symbol not found in MT5: {raw_symbol}")
        return None
    # ensure visible
    if not info.visible:
        try:
            mt5.symbol_select(raw_symbol, True)
        except Exception:
            pass
    count = window + 60  # a buffer
    bars = mt5.copy_rates_from_pos(raw_symbol, TF_MT5, 0, count)
    if bars is None or len(bars) < window + 2:
        print(f"❌ Not enough bars for {raw_symbol} (got {0 if bars is None else len(bars)})")
        return None
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')
    if 'tick_volume' in df.columns:
        df = df.rename(columns={'tick_volume': 'volume'})
    df = df[['open','high','low','close','volume']].copy()
    df['Close_raw'] = df['close']
    return df


In [8]:
# ================================================================
# ⚙️ Cell 4 - Auto-Fetch Live Bars for All Symbols
# ================================================================

# --------------------------------------
# Fetch parameters
# --------------------------------------
WINDOW = 50
BUFFER = 60
COUNT = WINDOW + BUFFER   # extra bars for safety

# --------------------------------------
# Fetch bars for each symbol individually
# --------------------------------------
def fetch_symbol_bars(symbol):
    """Fetch bars safely for one symbol."""
    # Ensure MT5 knows this symbol

    mt5.initialize()
    
    info = mt5.symbol_info(symbol)
    if info is None:
        print(f"❌ Symbol not found in MT5: {symbol}")
        return None

    if not info.visible:
        mt5.symbol_select(symbol, True)

    bars = mt5.copy_rates_from_pos(symbol, tf, 0, COUNT)

    if bars is None or len(bars) < WINDOW + 2:
        print(f"❌ Not enough bars for {symbol} (got {0 if bars is None else len(bars)})")
        return None

    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    mt5.shutdown()
    return df


# --------------------------------------
# Fetch all symbols into a dictionary
# --------------------------------------
symbol_data = {}

for sym in SYMBOLS:
    print(f"📡 Fetching live data for: {sym}")
    df = fetch_symbol_bars(sym)
    if df is not None:
        symbol_data[sym] = df
        print(f"✔️ Loaded {len(df)} bars for {sym}")
    else:
        print(f"⚠️ Failed to load data for {sym}")

print(f"\n✅ Completed fetching live data for {len(symbol_data)} symbols.")


📡 Fetching live data for: Volatility 10 Index
✔️ Loaded 110 bars for Volatility 10 Index
📡 Fetching live data for: Volatility 25 Index
✔️ Loaded 110 bars for Volatility 25 Index
📡 Fetching live data for: Volatility 50 Index
✔️ Loaded 110 bars for Volatility 50 Index
📡 Fetching live data for: Volatility 75 Index
✔️ Loaded 110 bars for Volatility 75 Index
📡 Fetching live data for: Volatility 100 Index
✔️ Loaded 110 bars for Volatility 100 Index
📡 Fetching live data for: Volatility 10 (1s) Index
✔️ Loaded 110 bars for Volatility 10 (1s) Index
📡 Fetching live data for: Volatility 25 (1s) Index
✔️ Loaded 110 bars for Volatility 25 (1s) Index
📡 Fetching live data for: Volatility 50 (1s) Index
✔️ Loaded 110 bars for Volatility 50 (1s) Index
📡 Fetching live data for: Volatility 75 (1s) Index
✔️ Loaded 110 bars for Volatility 75 (1s) Index
📡 Fetching live data for: Volatility 100 (1s) Index
✔️ Loaded 110 bars for Volatility 100 (1s) Index
📡 Fetching live data for: Volatility 10 (10s) Index
❌ Sy

In [9]:
# scalers: read per-asset CSVs created earlier (mean/std)
def load_scalers_from_csv(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                print("Skipping", p, "- unexpected format")
                continue
        if len(df) > window:
            datasets[safe] = df
    return datasets

def load_embeddings(embed_file=EMBED_FILE, data_dir=DATA_DIR):
    if not os.path.exists(embed_file):
        print("No embeddings file found:", embed_file)
        return {}
    emb = np.load(embed_file, allow_pickle=True)
    emb_dict = {}
    if os.path.exists(ASSET_MAP_FILE):
        am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
        for safe, idx in am.items():
            idx = int(idx)
            if idx < emb.shape[0]:
                emb_dict[safe] = np.array(emb[idx], dtype=np.float32)
    else:
        csvs = sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv")))
        safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
        if len(safe_list) == emb.shape[0]:
            for i, safe in enumerate(safe_list):
                emb_dict[safe] = np.array(emb[i], dtype=np.float32)
        else:
            print("Warning: embedding count and CSV count mismatch")
    return emb_dict

# Load them
scalers = load_scalers_from_csv(DATA_DIR)
datasets = load_datasets(DATA_DIR, WINDOW)
embeddings = load_embeddings(EMBED_FILE, DATA_DIR)

print("Loaded datasets:", len(datasets), "scalers:", len(scalers), "embeddings:", len(embeddings))
safe_list = sorted(datasets.keys())
print("Assets:", safe_list)


Loaded datasets: 16 scalers: 16 embeddings: 17
Assets: ['EURUSD', 'Jump_100_Index', 'Jump_10_Index', 'Jump_25_Index', 'Jump_50_Index', 'Jump_75_Index', 'Volatility_100_1s_Index', 'Volatility_100_Index', 'Volatility_10_1s_Index', 'Volatility_10_Index', 'Volatility_25_1s_Index', 'Volatility_25_Index', 'Volatility_50_1s_Index', 'Volatility_50_Index', 'Volatility_75_1s_Index', 'Volatility_75_Index']


In [10]:
# Load PPO and VecNormalize
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model file missing: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("Loaded PPO model:", MODEL_FILE)

vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE)
        print("Loaded VecNormalize:", VEC_FILE)
    except Exception as e:
        print("Warning: VecNormalize load failed:", e)
        vecnorm = None


Loaded PPO model: models\multiasset\ppo_multiasset.zip


In [24]:
# Cell: Safe VecNormalize loader using an inline dummy env (no external envs module required)
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# parameters that must match training
WINDOW = 50                   # your window used in training
N_PRICE_FEATURES = 5          # o_pc,h_pc,l_pc,c_pc,v_pc
EMBED_DIM = next(iter(embeddings.values())).shape[0] if len(embeddings)>0 else 8
OBS_DIM = N_PRICE_FEATURES + 1 + EMBED_DIM   # matches: 5 + 1 + embed_dim

class _DummyTradingEnv(gym.Env):
    """
    Minimal dummy env with the correct obs/action spaces for VecNormalize.load().
    Used only to provide a 'venv' for VecNormalize.load(path, venv).
    """
    def __init__(self):
        super().__init__()
        # observation: shape=(WINDOW, OBS_DIM)
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(WINDOW, OBS_DIM), dtype=np.float32)
        self.action_space = gym.spaces.Discrete(3)  # hold / buy / sell

    def reset(self, seed=None, options=None):
        # return an observation of correct shape and empty info dict
        obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        return obs, {}

    def step(self, action):
        # return obs, reward, done, truncated, info in gymnasium style
        obs = np.zeros(self.observation_space.shape, dtype=np.float32)
        reward = 0.0
        done = True   # immediate done is fine for dummy
        truncated = False
        info = {}
        return obs, reward, done, truncated, info

# Create vectorized dummy env with 1 env (can be >1)
venv = DummyVecEnv([lambda: _DummyTradingEnv()])

# Try loading VecNormalize with the dummy venv
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE, venv)
        # ensure it's in inference mode
        vecnorm.training = False
        vecnorm.norm_reward = False
        print("Loaded VecNormalize from", VEC_FILE)
    except Exception as e:
        print("Warning: VecNormalize.load failed:", e)
        vecnorm = None
else:
    print("No VecNormalize file found at", VEC_FILE)


Loaded VecNormalize from models\multiasset\vec_normalize.pkl


In [13]:
def fetch_and_build_obs(safe_name, window, scalers, embeddings, datasets, safe_list):
    """
    Build observation consistent with training:
    obs shape = (window, 5 + 1 + embed_dim)
    where 5 = o_pc,h_pc,l_pc,c_pc,v_pc ; 1 = balance (fixed 1.0) ; embed_dim from embeddings.
    """
    # prefer live bars, but fallback to preloaded dataset if fetch fails
    df_live = fetch_data_for_symbol(safe_name, window)
    if df_live is None:
        # fallback to prepared dataset
        if safe_name not in datasets:
            print("No data available for", safe_name)
            return None, None, None
        df = datasets[safe_name].iloc[-(window+1):].copy()
        # reconstruct OHLCV in price-space from Close_raw if needed is not possible; but we expect dataset to contain pct-change already
        # We'll use stored pct columns if present:
        df_pct = df[['o_pc','h_pc','l_pc','c_pc','v_pc']].copy()
        last_price = float(df['Close_raw'].iloc[-1])
    else:
        # compute pct-change from live OHLCV
        df = df_live
        df_pct = df[['open','high','low','close','volume']].pct_change().dropna()
        if len(df_pct) < window:
            print("Not enough pct rows for", safe_name)
            return None, None, None
        df_pct = df_pct.tail(window)
        last_price = float(df['Close_raw'].iloc[-1])

    # when using infile dataset, df_pct above already length==window
    if df_pct.shape[0] < window:
        print("Window mismatch for", safe_name)
        return None, None, None

    # get scaler
    if safe_name not in scalers:
        print("No scaler for", safe_name)
        return None, None, None
    s = scalers[safe_name]
    mean = np.array(s['mean'].values if hasattr(s['mean'], 'values') else s['mean'], dtype=np.float32)
    std  = np.array(s['std'].values if hasattr(s['std'], 'values') else s['std'], dtype=np.float32)
    std = np.where(std == 0, 1e-8, std)

    # Ensure df_pct column order matches mean/std indices: scalers were saved with index names o_pc,h_pc,l_pc,c_pc,v_pc
    cols = ['o_pc','h_pc','l_pc','c_pc','v_pc']
    # If df_pct currently has original OHLCV names, convert order
    if list(df_pct.columns) != cols:
        # attempt to rename if current names are open,high,...
        if all(c in df_pct.columns for c in ['open','high','low','close','volume']):
            df_pct = df_pct[['open','high','low','close','volume']]
        else:
            # assume df_pct already has o_pc,h_pc...
            pass

    # get features array of shape (window,5)
    features = df_pct[cols].values.astype(np.float32) if all(c in df_pct.columns for c in cols) else df_pct.values.astype(np.float32)
    features_scaled = (features - mean) / std

    # embedding
    if safe_name in embeddings:
        emb_vec = np.array(embeddings[safe_name], dtype=np.float32)
    else:
        emb_dim = next(iter(embeddings.values())).shape[0] if len(embeddings)>0 else 8
        emb_vec = np.zeros(emb_dim, dtype=np.float32)
    emb_block = np.repeat(emb_vec.reshape(1,-1), window, axis=0)

    # balance column (used in training) - keep as ones
    balance_col = np.ones((window,1), dtype=np.float32)

    obs = np.concatenate([features_scaled, balance_col, emb_block], axis=1).astype(np.float32)

    # If VecNormalize exists and matches expected flattened size, attempt to apply (vecnorm expects flattened obs in older SB3 versions)
    if vecnorm is not None:
        try:
            flat = obs.reshape(1, -1)
            mean_rms = getattr(vecnorm.obs_rms, "mean", None)
            var_rms = getattr(vecnorm.obs_rms, "var", None)
            if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                obs = flat_norm.reshape(obs.shape).astype(np.float32)
            else:
                # some SB3 versions provide normalize_obs
                try:
                    obs = vecnorm.normalize_obs(obs, False)
                except Exception:
                    pass
        except Exception as e:
            print("VecNormalize apply error:", e)

    # volatility estimate for SL/lot sizing
    vol_est = float(df_pct['c_pc'].std()) if 'c_pc' in df_pct.columns else float(np.std(features[:,3]))

    return obs, vol_est, last_price


In [14]:
def pip_value(symbol):
    return 0.01 if "JPY" in symbol.upper() else 0.0001

def estimate_sl_pips_from_vol(raw_symbol, last_price, vol_est, min_pips=5, max_pips=200):
    if "JPY" in raw_symbol.upper():
        pip = 0.01
    else:
        pip = 0.0001
    abs_move = vol_est * last_price
    if abs_move <= 0:
        sl_pips = DEFAULT_SL_PIPS_FALLBACK
    else:
        sl_raw = abs_move / pip
        sl_pips = float(np.clip(np.round(sl_raw * 1.5), min_pips, max_pips))
    return int(sl_pips)

def compute_lot_from_balance(balance, vol, price, risk_pct=0.005, min_lot=0.01, max_lot=1.0):
    risk_amount = balance * risk_pct
    vol = max(vol, 1e-8)
    price_scale = 1000.0
    lot = risk_amount / (vol * price_scale)
    return float(max(min_lot, min(max_lot, round(lot, 2))))

def compute_sl_tp_by_pips(symbol, price, direction, sl_pips, tp_pips):
    pip = pip_value(symbol)
    if direction == "BUY":
        sl = price - sl_pips * pip
        tp = price + tp_pips * pip
    else:
        sl = price + sl_pips * pip
        tp = price - tp_pips * pip
    return float(sl), float(tp)

def get_positions_for_symbol(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_market_order(symbol, direction, lot, sl_price=None, tp_price=None):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ Cannot get tick for", symbol)
        return {"retcode": None, "comment":"NO_TICK"}
    price = float(tick.ask if direction=="BUY" else tick.bid)
    if DRY_RUN:
        return {"retcode": 10009, "price": price, "comment":"DRY_RUN", "direction":direction}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    res = mt5.order_send(req)
    return res

def close_position_by_ticket(ticket):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p, "type", None) == 0:  # BUY => close with SELL
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [15]:
def run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets):
    header = not os.path.exists(LOG_FILE)
    acct = mt5.account_info()
    balance = float(acct.balance) if acct else 10000.0

    for safe_name in safe_list:
        raw_symbol = get_mt5_symbol(safe_name)
        print(f"\n--- {raw_symbol} ({safe_name}) ---")
        obs, vol, last_price = fetch_and_build_obs(safe_name, WINDOW, scalers, embeddings, datasets, safe_list)
        if obs is None:
            print("Skip", safe_name)
            continue

        try:
            action, _ = model.predict(obs[np.newaxis,...], deterministic=True)
            a = int(action[0]) if isinstance(action,(list,tuple,np.ndarray)) else int(action)
            print("Signal:", a)
        except Exception as e:
            print("Prediction error:", e)
            continue

        positions = get_positions_for_symbol(raw_symbol)
        print("Existing positions:", len(positions))

        # auto-close opposing
        if a == 1:
            for p in positions:
                if getattr(p,"type",None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:
            for p in positions:
                if getattr(p,"type",None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # refresh positions
        positions = get_positions_for_symbol(raw_symbol)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached, skipping open")
        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a==1 else "SELL"
                lot = compute_lot_from_balance(balance, vol, last_price)
                sl_pips = estimate_sl_pips_from_vol(raw_symbol, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)
                sl_price, tp_price = compute_sl_tp_by_pips(raw_symbol, last_price, direction, sl_pips, tp_pips)
                res = place_market_order(raw_symbol, direction, lot, sl_price, tp_price)

                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res,"retcode", None)
                    comment = getattr(res,"comment", "")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": raw_symbol,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", lot, "retcode", retcode)

        # trailing
        for p in get_positions_for_symbol(raw_symbol):
            # recompute vol-based trail
            trail_pips = max(5, int(estimate_sl_pips_from_vol(raw_symbol, last_price, vol) // 2))
            pip = pip_value(raw_symbol)
            if getattr(p,"type",None) == 0:  # BUY
                cur = mt5.symbol_info_tick(raw_symbol).bid
                new_sl = float(cur - trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw_symbol, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))
            else:  # SELL
                cur = mt5.symbol_info_tick(raw_symbol).ask
                new_sl = float(cur + trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw_symbol, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))

    print("\nSingle pass complete.")


In [16]:
def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        # ensure expected columns exist; otherwise try conversion as earlier notebooks did
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            # attempt auto-convert if original OHLCV present
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                raise ValueError(f"{p} missing expected columns and cannot convert")
        if len(df) > window:
            datasets[safe] = df
    return datasets


In [17]:
datasets = load_datasets(data_dir=DATA_DIR, window=WINDOW) 

In [18]:
safe_list = sorted(datasets.keys())
print("Assets:", safe_list)
run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets)


Assets: ['EURUSD', 'Jump_100_Index', 'Jump_10_Index', 'Jump_25_Index', 'Jump_50_Index', 'Jump_75_Index', 'Volatility_100_1s_Index', 'Volatility_100_Index', 'Volatility_10_1s_Index', 'Volatility_10_Index', 'Volatility_25_1s_Index', 'Volatility_25_Index', 'Volatility_50_1s_Index', 'Volatility_50_Index', 'Volatility_75_1s_Index', 'Volatility_75_Index']


NameError: name 'get_mt5_symbol' is not defined

In [ ]:
# Use with caution. Set DRY_RUN=False only after testing thoroughly.
# try:
#     while True:
#         run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets)
#         time.sleep(60)  # wait one minute (for M1)
# except KeyboardInterrupt:
#     print("Stopped by user")


In [ ]:
if os.path.exists(LOG_FILE):
    df = pd.read_csv(LOG_FILE)
    display(df.tail(30))
else:
    print("No logs yet at", LOG_FILE)


end

In [25]:
# Cell 4
# Load per-asset scalers saved as CSVs: {safe}_scaler.csv with columns mean/std
def load_scalers_from_csv(data_dir=DATA_DIR):
    scalers = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_scaler.csv"))):
        safe = os.path.basename(p).replace("_scaler.csv","")
        df = pd.read_csv(p, index_col=0)
        # ensure ordering of mean/std columns matches our feature order: o_pc,h_pc,l_pc,c_pc,v_pc
        scalers[safe] = {"mean": df["mean"], "std": df["std"]}
    return scalers

def load_datasets(data_dir=DATA_DIR, window=WINDOW):
    datasets = {}
    for p in sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv"))):
        safe = os.path.basename(p).replace("_normalized.csv","")
        df = pd.read_csv(p, index_col=0, parse_dates=True)
        # ensure expected columns exist; otherwise try conversion as earlier notebooks did
        expected = ['o_pc','h_pc','l_pc','c_pc','v_pc','Close_raw']
        if all(c in df.columns for c in expected):
            df = df[expected].dropna()
        else:
            # attempt auto-convert if original OHLCV present
            if all(c in df.columns for c in ['open','high','low','close','volume']):
                tmp = pd.DataFrame(index=df.index)
                tmp['o_pc'] = df['open'].pct_change()
                tmp['h_pc'] = df['high'].pct_change()
                tmp['l_pc'] = df['low'].pct_change()
                tmp['c_pc'] = df['close'].pct_change()
                tmp['v_pc'] = df['volume'].pct_change()
                tmp['Close_raw'] = df['close']
                df = tmp.dropna()
            else:
                raise ValueError(f"{p} missing expected columns and cannot convert")
        if len(df) > window:
            datasets[safe] = df
    return datasets

def load_embeddings(embed_file=EMBED_FILE, data_dir=DATA_DIR):
    if not os.path.exists(embed_file):
        print("No embeddings file found at", embed_file)
        return {}
    emb = np.load(embed_file, allow_pickle=True)
    # emb likely is ndarray shape (n_assets, embed_dim)
    # Map using asset_to_idx.csv if present
    emb_dict = {}
    if os.path.exists(ASSET_MAP_FILE):
        am = pd.read_csv(ASSET_MAP_FILE, index_col=0, header=None).iloc[:,0].to_dict()
        # am is mapping safe->idx
        for safe, idx in am.items():
            idx = int(idx)
            if idx < emb.shape[0]:
                emb_dict[safe] = np.array(emb[idx], dtype=np.float32)
    else:
        # fallback: map by order of normalized CSVs if sizes match
        csvs = sorted(glob.glob(os.path.join(data_dir, "*_normalized.csv")))
        safe_list = [os.path.basename(p).replace("_normalized.csv","") for p in csvs]
        if len(safe_list) == emb.shape[0]:
            for i, safe in enumerate(safe_list):
                emb_dict[safe] = np.array(emb[i], dtype=np.float32)
        else:
            print("Warning: embedding count and CSV count mismatch; manual mapping required.")
    return emb_dict

# load
scalers = load_scalers_from_csv(DATA_DIR)
datasets = load_datasets(DATA_DIR, WINDOW)
embeddings = load_embeddings(EMBED_FILE, DATA_DIR)

print("Loaded assets:", len(datasets))
print("Loaded scalers:", len(scalers))
print("Loaded embeddings:", len(embeddings))


Loaded assets: 16
Loaded scalers: 16
Loaded embeddings: 17


In [72]:
# Cell 5
# Load PPO model
if not os.path.exists(MODEL_FILE):
    raise FileNotFoundError("Model not found: " + MODEL_FILE)
model = PPO.load(MODEL_FILE)
print("Loaded PPO from", MODEL_FILE)

# Try to load VecNormalize (optional)
vecnorm = None
if os.path.exists(VEC_FILE):
    try:
        vecnorm = VecNormalize.load(VEC_FILE)
        print("Loaded VecNormalize from", VEC_FILE)
    except Exception as e:
        print("VecNormalize load failed:", e)
        vecnorm = None


Loaded PPO from models\multiasset\ppo_multiasset.zip
VecNormalize load failed: VecNormalize.load() missing 1 required positional argument: 'venv'


# ================================================================
# Cell 5 — fetch_and_build_obs for live trading (fixed)
# ================================================================
def fetch_and_build_obs(safe_name, window, scalers, embeddings, safe_list):
    """
    safe_name: e.g., 'EURUSD', 'Jump_100_Index'
    Returns:
        obs: np.array (window, n_features + embedding + extras)
        vol: float, estimated volatility
        last_price: float
    """
    # Map safe_name -> broker symbol
    raw_symbol = get_mt5_symbol(safe_name)

    # Fetch latest bars
    df = fetch_data_for_symbol(safe_name, window)
    if df is None:
        return None, None, None

    # Use pct-change features
    df_pct = df[['open','high','low','close','volume']].pct_change().dropna()
    if len(df_pct) < window:
        print(f"❌ Not enough pct rows for {safe_name}")
        return None, None, None
    df_pct = df_pct.tail(window)

    # Scale features
    scaler = scalers[safe_name]
    mean = scaler['mean'][['o_pc','h_pc','l_pc','c_pc','v_pc']]
    std  = scaler['std'][['o_pc','h_pc','l_pc','c_pc','v_pc']].replace(0,1.0)
    features_scaled = (df_pct.values - mean.values) / std.values

    # Embedding
    embed = embeddings.get(safe_name, np.zeros((list(embeddings.values())[0].shape[0],), dtype=np.float32))
    emb_rep = np.repeat(embed[np.newaxis,:], window, axis=0)

    # Extra columns: balance_norm=1.0, asset_id normalized
    balance_norm = np.ones((window,1), dtype=np.float32)
    asset_id_val = safe_list.index(safe_name)/max(1,len(safe_list))
    asset_id_col = np.full((window,1), asset_id_val, dtype=np.float32)

    # Final observation
    obs = np.hstack([features_scaled.astype(np.float32), emb_rep, balance_norm, asset_id_col])
    last_price = float(df['Close_raw'].iloc[-1])
    vol = float(df_pct['close'].std())

    return obs, vol, last_price


In [74]:
# Cell 6
def fetch_and_build_obs(raw_symbol, window, scalers, embeddings, datasets, safe_list):
    """
    Build observation matching training:
    obs shape = (window, n_features=5 + 1 + embed_dim)
      -> 5: o_pc,h_pc,l_pc,c_pc,v_pc
      -> 1: volatility column (c_pc) per timestep (training used balance column previously; we use c_pc as volatility column)
         NOTE: original training used 'balance' column as +1; earlier env used balance=1.0. To be exact, we will include 'balance' column instead of vol column so obs_dim matches training:
         training obs was [OHLCV_pctnorm, balance, embedding]. So we'll include balance (1.0) and embedding; volatility used for SL only.
    """
    safe = safe_from_raw(raw_symbol)
    if safe not in datasets:
        print("No dataset for", safe)
        return None, None, None
    df = datasets[safe]
    if len(df) < window:
        print("Not enough data for", safe)
        return None, None, None

    window_df = df.iloc[-window:]
    # features (pct-changes)
    try:
        features = window_df[['o_pc','h_pc','l_pc','c_pc','v_pc']].values.astype(np.float32)  # shape (window,5)
        last_price = float(window_df['Close_raw'].iloc[-1])
    except KeyError as e:
        print("Column missing:", e)
        return None, None, None

    # scale using scalers dict (mean/std may be pandas Series)
    if safe not in scalers:
        print("No scaler for", safe)
        return None, None, None
    s = scalers[safe]
    # accept either pandas Series or numpy arrays
    mean = np.array(s['mean'].values if hasattr(s['mean'], "values") else s['mean'], dtype=np.float32)
    std  = np.array(s['std'].values if hasattr(s['std'], "values") else s['std'], dtype=np.float32)
    std = np.where(std == 0, 1e-8, std)
    features_scaled = (features - mean) / std

    # embedding vector
    if safe in embeddings:
        emb_vec = np.array(embeddings[safe], dtype=np.float32)
    else:
        # fallback zero vector of expected embedding dim (use first embedding length if possible)
        if len(embeddings)>0:
            emb_dim = next(iter(embeddings.values())).shape[0]
            emb_vec = np.zeros(emb_dim, dtype=np.float32)
        else:
            emb_vec = np.zeros(8, dtype=np.float32)  # fallback to 8

    emb_block = np.repeat(emb_vec.reshape(1, -1), window, axis=0)  # (window, embed_dim)

    # Balance column (training used balance; set to 1.0)
    balance_col = np.full((window,1), 1.0, dtype=np.float32)

    # Final obs: [features_scaled (5) , balance (1) , embedding (embed_dim)]
    obs = np.concatenate([features_scaled, balance_col, emb_block], axis=1).astype(np.float32)  # (window, 5+1+embed_dim)

    # VecNormalize optional normalization: apply if present (flatten-first)
    if vecnorm is not None:
        try:
            # vecnorm.obs_rms.mean shape matches flattened obs; we need to flatten timestep dimension
            flat = obs.reshape(1, -1)  # shape (1, window * feat)
            mean_rms = getattr(vecnorm.obs_rms, "mean", None)
            var_rms = getattr(vecnorm.obs_rms, "var", None)
            if mean_rms is not None and var_rms is not None and len(mean_rms) == flat.shape[1]:
                flat_norm = (flat - mean_rms) / np.sqrt(var_rms + 1e-8)
                obs = flat_norm.reshape(obs.shape).astype(np.float32)
            else:
                # fallback: use vecnorm.normalize_obs if available (some versions)
                try:
                    obs = vecnorm.normalize_obs(obs, False)
                except Exception:
                    pass
        except Exception as e:
            print("VecNormalize apply failed:", e)

    # estimate vol for lot/SL computation (std of c_pc)
    vol_est = float(window_df['c_pc'].std())

    return obs, vol_est, last_price


In [75]:
# Cell 7
def estimate_sl_pips_from_vol(raw_symbol, last_price, vol_est, min_pips=5, max_pips=200):
    """
    Heuristic placeholder for 'learned' SL:
    - vol_est is std of c_pc (pct-change) per bar
    - approximate absolute move = vol_est * last_price
    - convert to pips: pip = 0.0001 (or 0.01 for JPY)
    - produce sl_pips clipped to [min_pips, max_pips]
    """
    if "JPY" in raw_symbol.upper():
        pip = 0.01
    else:
        pip = 0.0001
    # absolute expected move:
    abs_move = vol_est * last_price
    if abs_move <= 0:
        sl_pips = DEFAULT_SL_PIPS_FALLBACK
    else:
        sl_raw = abs_move / pip
        # safety scale factor
        sl_pips = float(np.clip(np.round(sl_raw * 1.5), min_pips, max_pips))
    return int(sl_pips)

def compute_lot_fixed(default_lot=DEFAULT_LOT):
    return float(default_lot)


In [76]:
# Cell 8
def get_positions_for_symbol(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else list(pos)

def place_market_order(symbol, direction, lot, sl_price=None, tp_price=None):
    """
    direction: "BUY" or "SELL"
    returns dict-like result (simulated if DRY_RUN)
    """
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        print("❌ Cannot get tick for", symbol)
        return None
    price = float(tick.ask if direction=="BUY" else tick.bid)
    if DRY_RUN:
        # simulated response
        return {"retcode": 10009, "price": price, "comment":"DRY_RUN", "direction":direction}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(lot),
        "type": mt5.ORDER_TYPE_BUY if direction=="BUY" else mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": float(sl_price) if sl_price is not None else 0.0,
        "tp": float(tp_price) if tp_price is not None else 0.0,
        "deviation": 20,
        "magic": 234000,
        "comment": "ppo_multiasset_live",
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    res = mt5.order_send(req)
    return res

def close_position_by_ticket(ticket):
    pos_list = mt5.positions_get(ticket=ticket)
    if not pos_list:
        return None
    p = pos_list[0]
    symbol = p.symbol
    if getattr(p, "type", None) == 0:  # BUY => close with SELL
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    if DRY_RUN:
        return {"retcode":10009, "comment":"DRY_RUN_CLOSE", "ticket": ticket}
    req = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(p.volume),
        "type": order_type,
        "position": int(ticket),
        "price": price,
        "deviation": 20,
        "magic": 234000,
        "comment": "auto_close",
    }
    return mt5.order_send(req)


In [77]:
# Cell 9
def run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets):
    header = not os.path.exists(LOG_FILE)
    acct = mt5.account_info()
    balance = float(acct.balance) if acct else 10000.0

    # iterate assets
    for safe in safe_list:
        raw = raw_from_safe(safe)   # MT5 symbol name guess
        print("\n---", raw, "(", safe, ") ---")
        obs, vol, last_price = fetch_and_build_obs(raw, WINDOW, scalers, embeddings, datasets, safe_list)
        if obs is None:
            print("Skip", raw)
            continue

        # model expects batch dim
        try:
            action, _ = model.predict(obs[np.newaxis, ...], deterministic=True)
            a = int(action[0]) if isinstance(action, (list,tuple,np.ndarray)) else int(action)
        except Exception as e:
            print("Prediction error:", e)
            continue

        print("Signal:", a, " (0=HOLD,1=BUY,2=SELL)")

        # get current positions for symbol
        positions = get_positions_for_symbol(raw)
        print("Existing positions:", len(positions))

        # Close opposing positions if reversal
        if a == 1:
            # BUY signal -> close SELL positions (type==1)
            for p in positions:
                if getattr(p,"type",None) == 1:
                    print("Closing opposing SELL ticket", p.ticket)
                    close_position_by_ticket(p.ticket)
        elif a == 2:
            for p in positions:
                if getattr(p,"type",None) == 0:
                    print("Closing opposing BUY ticket", p.ticket)
                    close_position_by_ticket(p.ticket)

        # refresh positions
        positions = get_positions_for_symbol(raw)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print("Max positions reached for", raw)
        else:
            if a == 0:
                print("HOLD — no trade")
            else:
                direction = "BUY" if a==1 else "SELL"
                lot = compute_lot_fixed(DEFAULT_LOT)
                # estimate SL pips from volatility, then compute TP = 3 * SL
                sl_pips = estimate_sl_pips_from_vol(raw, last_price, vol)
                tp_pips = int(sl_pips * TP_MULT)
                # convert pips to price levels
                if "JPY" in raw.upper():
                    pip = 0.01
                else:
                    pip = 0.0001
                if direction == "BUY":
                    sl_price = last_price - sl_pips * pip
                    tp_price = last_price + tp_pips * pip
                else:
                    sl_price = last_price + sl_pips * pip
                    tp_price = last_price - tp_pips * pip

                res = place_market_order(raw, direction, lot, sl_price, tp_price)
                # normalize response
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    exec_price = res.get("price", last_price)
                else:
                    retcode = getattr(res, "retcode", None)
                    comment = getattr(res, "comment", "")
                    exec_price = last_price

                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe,
                    "symbol": raw,
                    "action": direction,
                    "lot": lot,
                    "exec_price": exec_price,
                    "sl_price": sl_price,
                    "tp_price": tp_price,
                    "sl_pips": sl_pips,
                    "tp_pips": tp_pips,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print("Placed", direction, "lot", lot, "retcode", retcode)

        # Trailing: update SL for existing positions if profitable (simple rule)
        for p in get_positions_for_symbol(raw):
            # compute trailing SL target in price; use trailing pips = max(5, sl_pips//2) as example
            try:
                trail_pips = max(5, int(estimate_sl_pips_from_vol(raw, last_price, vol) // 2))
            except Exception:
                trail_pips = 5
            if "JPY" in raw.upper():
                pip = 0.01
            else:
                pip = 0.0001
            if getattr(p,"type",None) == 0:  # BUY
                # new SL = current price - trail_pips
                cur = mt5.symbol_info_tick(raw).bid
                new_sl = float(cur - trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))
            else:  # SELL
                cur = mt5.symbol_info_tick(raw).ask
                new_sl = float(cur + trail_pips * pip)
                if DRY_RUN:
                    print(f"[DRY] Would set trailing SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {"action": mt5.TRADE_ACTION_SLTP, "symbol": raw, "position": int(p.ticket), "sl": new_sl, "tp": float(p.tp) if getattr(p,"tp",None) else 0.0}
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode",None))

    print("\nSingle pass complete.")


# ================================================================
# Cell 8 — run_once_predict_and_manage (fixed)
# ================================================================
def run_once_predict_and_manage(model, safe_list, scalers, embeddings):
    """
    Loop over all assets, predict, place orders, auto-close on reversal,
    enforce MAX_POS_PER_SYMBOL, apply trailing SL.
    """
    # Account info
    acct = mt5.account_info()
    balance = float(acct.balance) if acct else 10000.0

    header = not os.path.exists(LOG_FILE)

    for safe_name in safe_list:
        raw_symbol = get_mt5_symbol(safe_name)
        print(f"\n--- {raw_symbol} ({safe_name}) ---")

        obs, vol, last_price = fetch_and_build_obs(safe_name, WINDOW, scalers, embeddings, safe_list)
        if obs is None:
            print(f"Skip {safe_name}")
            continue

        # Predict
        try:
            action, _ = model.predict(obs[np.newaxis,...], deterministic=True)
            a = int(action[0]) if isinstance(action,(list,tuple,np.ndarray)) else int(action)
            print("Signal:", a, "(0=HOLD,1=BUY,2=SELL)")
        except Exception as e:
            print("Prediction error:", e)
            continue

        # Get current positions
        positions = get_positions_for_symbol(raw_symbol)
        print("Existing positions:", len(positions))

        # Auto-close opposing positions
        if a == 1:
            for p in positions:
                if getattr(p, "type", None) == 1:  # SELL
                    close_position_by_ticket(p.ticket)
        elif a == 2:
            for p in positions:
                if getattr(p, "type", None) == 0:  # BUY
                    close_position_by_ticket(p.ticket)

        # Re-fetch positions & enforce max
        positions = get_positions_for_symbol(raw_symbol)
        if len(positions) >= MAX_POS_PER_SYMBOL:
            print(f"Max positions reached for {raw_symbol} ({len(positions)})")
        else:
            if a == 0:
                print("HOLD")
            else:
                direction = "BUY" if a==1 else "SELL"
                lot = compute_lot_from_balance(balance, vol, last_price)
                sl, tp = compute_sl_tp_by_pips(raw_symbol, last_price, direction, DEFAULT_SL_PIPS, DEFAULT_TP_PIPS)
                res = place_market_order(raw_symbol, direction, lot, sl, tp)
                # Log trade
                if isinstance(res, dict):
                    retcode = res.get("retcode")
                    comment = res.get("comment")
                    price_executed = res.get("price", last_price)
                else:
                    retcode = getattr(res,"retcode",None)
                    comment = getattr(res,"comment","")
                    price_executed = last_price
                entry = {
                    "timestamp": datetime.utcnow().isoformat(),
                    "safe": safe_name,
                    "symbol": raw_symbol,
                    "action": direction,
                    "lot": lot,
                    "exec_price": price_executed,
                    "sl": sl,
                    "tp": tp,
                    "retcode": retcode,
                    "comment": comment,
                    "dry_run": DRY_RUN
                }
                pd.DataFrame([entry]).to_csv(LOG_FILE, mode="a", index=False, header=header)
                header = False
                print(f"Placed {direction} lot {lot} retcode {retcode}")

        # Trailing SL
        for p in get_positions_for_symbol(raw_symbol):
            new_sl = trailing_sl_level(raw_symbol, p, TRAIL_PIPS)
            if new_sl:
                if DRY_RUN:
                    print(f"[DRY] Would modify SL for ticket {p.ticket} -> {new_sl}")
                else:
                    req = {
                        "action": mt5.TRADE_ACTION_SLTP,
                        "symbol": raw_symbol,
                        "position": int(p.ticket),
                        "sl": new_sl,
                        "tp": float(p.tp) if getattr(p,"tp",None) else 0.0
                    }
                    r = mt5.order_send(req)
                    print("Modify SL result:", getattr(r,"retcode", None))

    print("\nSingle pass complete.")


In [78]:
# Cell 10
safe_list = sorted(datasets.keys())
print("Assets to process:", safe_list)
run_once_predict_and_manage(model, safe_list, scalers, embeddings, datasets)


Assets to process: ['EURUSD', 'Jump_100_Index', 'Jump_10_Index', 'Jump_25_Index', 'Jump_50_Index', 'Jump_75_Index', 'Volatility_100_1s_Index', 'Volatility_100_Index', 'Volatility_10_1s_Index', 'Volatility_10_Index', 'Volatility_25_1s_Index', 'Volatility_25_Index', 'Volatility_50_1s_Index', 'Volatility_50_Index', 'Volatility_75_1s_Index', 'Volatility_75_Index']

--- EURUSD ( EURUSD ) ---
Signal: 2  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for EURUSD
Placed SELL lot 0.1 retcode None

--- Jump 100 Index ( Jump_100_Index ) ---
Signal: 1  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Jump 100 Index
Placed BUY lot 0.1 retcode None

--- Jump 10 Index ( Jump_10_Index ) ---
Signal: 1  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Jump 10 Index
Placed BUY lot 0.1 retcode None

--- Jump 25 Index ( Jump_25_Index ) ---
Signal: 0  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD — no trade

--- Jump 50 Index ( Jump_50_Index ) ---
Signal: 1

C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712

Signal: 1  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Jump 75 Index
Placed BUY lot 0.1 retcode None

--- Volatility 100 1s Index ( Volatility_100_1s_Index ) ---
Signal: 2  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Volatility 100 1s Index
Placed SELL lot 0.1 retcode None

--- Volatility 100 Index ( Volatility_100_Index ) ---
Signal: 2  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Volatility 100 Index
Placed SELL lot 0.1 retcode None

--- Volatility 10 1s Index ( Volatility_10_1s_Index ) ---
Signal: 2  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Volatility 10 1s Index
Placed SELL lot 0.1 retcode None

--- Volatility 10 Index ( Volatility_10_Index ) ---
Signal: 2  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
❌ Cannot get tick for Volatility 10 Index
Placed SELL lot 0.1 retcode None

--- Volatility 25 1s Index ( Volatility_25_1s_Index ) ---
Signal: 0  (0=HOLD,1=BUY,2=SELL)
Existing positions: 0
HOLD — 

C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712\2806280530.py:80: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat(),
C:\Users\Slick\AppData\Local\Temp\ipykernel_11712

In [40]:
# Cell 11 (optional)
def simulate_trades_on_history(datasets, model, assets=None, window=WINDOW, horizon=10):
    trades = []
    assets = assets or list(datasets.keys())
    for safe in assets:
        df = datasets[safe].reset_index(drop=True)
        emb = embeddings.get(safe, np.zeros((0,)))
        for i in range(window, len(df)-horizon):
            window_df = df.iloc[i-window:i]
            feat = window_df[['o_pc','h_pc','l_pc','c_pc','v_pc']].values.astype(np.float32)
            # scale manually as above
            mean = scalers[safe]['mean'].values
            std = scalers[safe]['std'].values
            std = np.where(std==0,1e-8,std)
            feat_scaled = (feat - mean) / std
            emb_rep = np.tile(emb.reshape(1,-1),(window,1)) if emb.size>0 else np.zeros((window,0))
            balance_col = np.full((window,1),1.0,dtype=np.float32)
            obs = np.concatenate([feat_scaled, balance_col, emb_rep], axis=1)
            try:
                action, _ = model.predict(obs[np.newaxis,...], deterministic=True)
                action = int(action[0]) if isinstance(action,(list,tuple,np.ndarray)) else int(action)
            except Exception:
                continue
            if action == 0: continue
            entry_price = float(df['Close_raw'].iat[i-1])
            exit_price = float(df['Close_raw'].iat[i+horizon-1])
            pos = 1 if action==1 else -1
            pnl = (exit_price - entry_price)/entry_price * pos
            trades.append({"safe": safe, "entry_i": i, "horizon": horizon, "action":action, "pnl":pnl})
    return pd.DataFrame(trades)

# run quick sim
print("Running quick simulated backtest (may take a while)...")
df_trades = simulate_trades_on_history(datasets, model, assets=safe_list, window=WINDOW, horizon=10)
print("Simulated trades:", len(df_trades))
if not df_trades.empty:
    print("Total PnL:", df_trades['pnl'].sum(), "Win rate:", (df_trades['pnl']>0).mean())


Running quick simulated backtest (may take a while)...


KeyboardInterrupt: 

In [ ]:
# Cell 12
if os.path.exists(LOG_FILE):
    df_log = pd.read_csv(LOG_FILE)
    display(df_log.tail(20))
else:
    print("No live logs yet at", LOG_FILE)
